# Создание и анализ графового набора данных для проверки контрагентов с помощью веб-скрэйпинга или анализа открытой базы данных

## Парсинг информации о контрагентах с сервиса проверки и анализа контрагентов "Компаниум"

### Первый этап парсинга - сбор ссылок, в которых содержится ОГРН, со страницы "Расширенный поиск" на индивидуальные страницы организаций 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from bs4 import BeautifulSoup
import re
import random

Функция parse_companium собирает ссылки на компании со страниц расширенного поиска.

In [2]:
def parse_companium(html):
    soup = BeautifulSoup(html, 'html.parser')
    ogrn_elements = soup.find_all('td')
    ogrn = [i.find_all('a') for i in ogrn_elements][1::2]
    ogrn = [i[0]['href'] for i in ogrn]
    return ogrn

In [3]:
browser = webdriver.Chrome()
browser.get('https://companium.ru/advanced/search')
# Кнопка фильтра "Только действующие" 
button_1 = browser.find_element(By.XPATH, "//*[@id='vue']/div/div[4]/div/div/div/div/div[1]")
button_1.click()

# Кнопка выбора ИП
# button= browser.find_element(By.XPATH, '//*[@id="vue"]/div/div[1]/div/div/div[1]/div/div[2]/div/div')
# button.click()

In [ ]:
n_pages = 2000
all_ogrns_u = []
for i in range(n_pages): 
    time.sleep(0.85)
    html_content = browser.page_source
    ogrns = parse_companium(html_content)
    all_ogrns_u.extend(ogrns)
    button_next = browser.find_element(By.CSS_SELECTOR, "a[rel='next']")
    button_next.click()
print(len(all_ogrns_u))

In [ ]:
df = pd.DataFrame({'OGRN':all_ogrns_u})
df.to_csv('20501-22502pages.csv')

### Второй этап парсинга - сбор данных о компаниях с использованием ранее собранных ссылок на организации

Функция random_delay задает определенное количесво секунду в интервале от 9 до 12 - задержка между обработкой каждого сайта(компании). Это сделано в целях избежания блокировки и появления CAPTCHA.

In [8]:
def random_delay():    
    delay = random.uniform(9, 12)
    time.sleep(delay)

In [2]:
df = pd.read_csv('2-20501pages.csv', index_col=0)
sites = list(df['OGRN'])
sites[:5]

['/id/1071102001651-gazprom-pererabotka',
 '/id/1077758081664-atomehnergoprom',
 '/id/1077760061862-po-klub-mkm',
 '/id/1079847085966-osk',
 '/id/1095406025307-pik-garantiya-pribyli']

In [5]:
sites_testing = sites[11597:15000]
sites_testing 

['/id/1142543016802-vega',
 '/id/1156658107462-zheldorstroy',
 '/id/1157746496555-sc-ttm',
 '/id/1237700432727-ehreteum-som',
 '/id/1027708003256-neftegazinkor',
 '/id/1056604793057-ehnergosk',
 '/id/1103017001856-aspo',
 '/id/1114025001078-niarmedik-farma',
 '/id/1145001002695-medtekh',
 '/id/1177746619863-inzhtekhzashchita',
 '/id/1202300004642-sab-konstrakshen',
 '/id/1203900003141-pyaty-ehlement',
 '/id/1026201074657-tyazhpressmash',
 '/id/1073667037620-sz-dsk-1',
 '/id/1107536006742-zabdorstroy',
 '/id/1117017000473-global-market',
 '/id/1187847388079-sz-kvs-lyubograd',
 '/id/1027402701941-saturn-2',
 '/id/1036906002693-stroyresurs',
 '/id/1037821030708-npo-pigment',
 '/id/1078604002663-tdk',
 '/id/1097746431903-strim',
 '/id/1101690062638-dzhidisi-servisez',
 '/id/1107415002078-gird-avtofurgon',
 '/id/1136315000469-samrehk-ehkspluataciya',
 '/id/1157746679584-vrk',
 '/id/1192536018751-dalnevostochnaya-korporaciya-sinsin',
 '/id/1207700116414-evraz-stil-bilding',
 '/id/51677464446

Создаем датафрейм, в который будем сохранять данные о компании 

In [12]:
data = pd.DataFrame(columns=['name_of_company','ogrn','active','inn','kpp',
                             'okpo','reg','adress',
                             'act','act_code','ceo','ceo_inn',
                             'uch','uch_inn', 
                             'cn_ceo', 'cn_uch', 'cn_org', 'cn_kid', 'cn_pred', 'cn_fut',
                             'z_names', 'z_ogrns','p_names', 'p_ogrns',
                             'compet_name', 'compet_ogrn'])
len(data.columns)

26

In [13]:
browser = webdriver.Chrome()  

После открытия браузера, с помощью цикла поочередно проходим по каждой компании и с помощью BeautifulSoup собираем информацию о ней.

In [17]:
for index,s in enumerate(sites_testing):    
    random_delay()
    print(s)
    
    browser.get('https://companium.ru'+s)
    
    h = browser.page_source
    soup = BeautifulSoup(h, 'html.parser')
    
    #инн
    inn = soup.find_all('span', id='copy-inn-details')[0].text
    
    #огрн
    ogrn = int(re.findall(r'[0-9]+',s)[0])
    name_of_company = soup.find('h1', class_='mb-2').text
    
    #кпп
    kpp = soup.find_all('span', id='copy-kpp-details')[0].text
    
    #окпо
    try:
        okpo = soup.find_all('span', id='copy-okpo-details')[0].text
    except:
        okpo  = None

    #Дата регистрации
    reg_sent = soup.find_all('p', class_="mt-3 mb-0")[0].text
    reg_all = re.findall(r'\b(\d{1,2} \w+ \d{4})\b', reg_sent)
    reg = reg_all[0]
    
    #адрес 
    adress = soup.find('span', id='copy-address').text
    
    #действующая компания
    try:
        active = soup.find('div', class_='text-success fw-bold').text
    except:
        try:
            active = soup.find('div', class_='fw-bold special-status').text
        except: 
            try:
                active = soup.find('div', class_='fw-bold text-danger').text
            except: 
                active = soup.find('div', class_='text-danger fw-bold').text

    #вид деятельности
    try:
        act = soup.find_all('td', class_='w-100')[0].find('a').text
        act_code = [i.find_all('a')[0]['href'][-6:] for i in soup.find_all('td', class_='w-100')]
    except: 
        act, act_code = None,None

    #руководитель
    try:
        ceo = soup.find_all('div', class_='flex-grow-1 ms-3')[0].find('a').text
        ceo_inn = soup.find_all('div', class_='flex-grow-1 ms-3')[0].find('a')['href']
    except:
        ceo, ceo_inn = None,None
    
    #учредитель
    try:
        uch = soup.find_all('table', class_='table table-md -mt-2')[0].find_all('a')
        uch_inn = [int(re.findall(r'[0-9]+',i['href'])[0]) for i in uch]
        uch_names = [i.text for i in uch]
    except: 
        uch_names = None
        uch_inn = None

    #раздел связи
    try:
        # по руководителю
        cn_ceo = [i['href'] for i in soup.find_all('div', id="flush-collapse-1")[0].find_all('a')[:-1:2]]
        # по учредителю
        cn_uch = [i['href'] for i in soup.find_all('div', id="flush-collapse-2")[0].find_all('a')[:-1:2]]
        # управляемые организации
        cn_org = [i['href'] for i in soup.find_all('div', id="flush-collapse-3")[0].find_all('a')]
        # дочерние организации
        cn_kid = [i['href'] for i in soup.find_all('div', id="flush-collapse-4")[0].find_all('a')]
        # правопредшественники
        cn_pred = [i['href'] for i in soup.find_all('div', id="flush-collapse-5")[0].find_all('a')]
        # правопреемники
        cn_fut = [i['href'] for i in soup.find_all('div', id="flush-collapse-6")[0].find_all('a')]
    except:
        cn_ceo, cn_uch, cn_org, cn_kid, cn_pred, cn_fut = None,None,None,None,None,None

    # заказчики и поставщики
    try:
        zakazchiki = [i.find('a') for i in soup.find('div', id='contracts-tab-2').find_all('li')]
        z_names = [i.text for i in zakazchiki]
        z_ogrns = [int(re.findall(r'[0-9]+',i['href'])[0]) for i in zakazchiki]
    except:
        z_names, z_ogrns = None, None
    try:
        postavshiki = [i.find('a') for i in soup.find('div', id='contracts-tab-1').find_all('li')]
        p_names = [i.text for i in postavshiki]
        p_ogrns = [int(re.findall(r'[0-9]+',i['href'])[0]) for i in postavshiki]
    except:
        p_names, p_ogrns = None, None
        
    #конкуренты
    for i in soup.find_all('div', class_='col-12 col-md-6'):
        if i.find('strong') == None:
            compet_name = None
            compet_ogrn = None
        elif 'конкуренты' in i.find('strong').text.lower():
            compet = i.find_all('a')
            compet_name = [j.text for j in compet]
            compet_ogrn = [int(re.findall(r'[0-9]+',k['href'])[0]) for k in compet]
        else:
            compet_name = None
            compet_ogrn = None

    data.loc[index] = [name_of_company, ogrn, active, inn, kpp, okpo, reg, adress, act, act_code,
                        ceo, ceo_inn, uch_names, uch_inn, cn_ceo, cn_uch, cn_org, cn_kid, 
                        cn_pred, cn_fut, z_names, z_ogrns, p_names, p_ogrns,
                        compet_name, compet_ogrn]

/id/1142543016802-vega
/id/1156658107462-zheldorstroy
/id/1157746496555-sc-ttm
/id/1237700432727-ehreteum-som
/id/1027708003256-neftegazinkor
/id/1056604793057-ehnergosk
/id/1103017001856-aspo
/id/1114025001078-niarmedik-farma
/id/1145001002695-medtekh
/id/1177746619863-inzhtekhzashchita
/id/1202300004642-sab-konstrakshen
/id/1203900003141-pyaty-ehlement
/id/1026201074657-tyazhpressmash
/id/1073667037620-sz-dsk-1
/id/1107536006742-zabdorstroy
/id/1117017000473-global-market
/id/1187847388079-sz-kvs-lyubograd
/id/1027402701941-saturn-2
/id/1036906002693-stroyresurs
/id/1037821030708-npo-pigment
/id/1078604002663-tdk
/id/1097746431903-strim
/id/1101690062638-dzhidisi-servisez
/id/1107415002078-gird-avtofurgon
/id/1136315000469-samrehk-ehkspluataciya
/id/1157746679584-vrk
/id/1192536018751-dalnevostochnaya-korporaciya-sinsin
/id/1207700116414-evraz-stil-bilding
/id/5167746444674-gruppa-vis-ao
/id/1022300641186-pkf-dtk
/id/1027739558296-okdc-pao-gazprom
/id/1032305684124-mup-kttu
/id/10650

In [ ]:
data

,name_of_company,ogrn,active,inn,kpp,okpo,reg,adress,act,act_code,...,cn_org,cn_kid,cn_pred,cn_fut,p_names,p_ogrns,z_names,z_ogrns,compet_name,compet_ogrn
0,"ООО ""ВЕГА""",1142543016802,Действующая компания,2543055422,254301001,28808956,29 сентября 2014,"690049, Приморский край, г. о. Владивостокский...","Торговля оптовая лесоматериалами, строительным...","[467300, 453100, 453110, 453229, 461300, 46150...",...,None,None,None,None,None,None,None,None,"[ООО ""КОНТРОЛЬЛАЙН"", ООО ""АКСИОМА"", ООО ""СТРОЙ...","[1224800005385, 1217700104709, 1155012000890, ..."
1,"ООО ""ЖЕЛДОРСТРОЙ""",1156658107462,Юридическим лицом принято решение об изменении...,6679084643,667901001,45593569,25 декабря 2015,"620076, Свердловская область, г. о. город Екат...",Строительство жилых и нежилых зданий,"[412000, 421100, 421200, 421300, 422100, 42221...",...,[],"[/id/1197536001288-ehnergobamstroy, /id/113668...",[],[],"[ОАО ""ЭЛТЕЗА"", ОАО ""РЖД""]","[1057746620535, 1037739877295]",[],[],"[ООО ""КАРКАС МОНОЛИТ"", ООО ""СПЕЦИАЛИЗИРОВАННЫЙ...","[1037739419960, 1187847273360, 1027739623988, ..."
2,"ООО ""СЦ ТТМ""",1157746496555,Действующая компания,7709459534,770901001,66982430,3 июня 2015,"105005, г. Москва, вн. тер. г. муниципальный о...",Торговля оптовая производственным электротехни...,"[466950, 257200, 259400, 261100, 262000, 26300...",...,[],"[/id/1127746475845-multipas, /id/1177746613263...",[/id/1107746591622-ttm],[],"[ГУП ""МОСГОРТРАНС"", ГУП ""МОСКОВСКИЙ МЕТРОПОЛИТ...","[1037739376223, 1027700096280, 1027301172580, ...",[],[],"[ООО ""ИН ХОУМ"", ООО ""МИЛАНДР ЭК"", ООО ""ТОРГОВЫ...","[1192536029938, 1117746552494, 1031616011536, ..."
3,"ООО ""ЭРЕТЕУМ.СОМ""",1237700432727,Действующая компания,9715454030,771501001,54355159,26 июня 2023,"127204, г. Москва, вн. тер. г. муниципальный о...",Деятельность головных офисов,"[701000, 182000, 611030, 612000, 619000, 62010...",...,[],[],[],[],None,None,None,None,"[ГРУППА ""ВИС"" (АО), ООО УК ""АВГУСТ-АГРО"", ООО ...","[5167746444674, 1207700133662, 1157847375311, ..."
4,"АО ""НЕФТЕГАЗИНКОР""",1027708003256,Действующая компания,7708201998,770101001,59093675,23 августа 2002,"101000, г. Москва, вн. тер. г. муниципальный о...",Консультирование по вопросам коммерческой деят...,"[702200, 461200, 466900, 467100]",...,[],[/id/1027739272307-zn-ehkspertiza],[],[],"[МИНЭНЕРГО МОСКОВСКОЙ ОБЛАСТИ, ЦЕНТРАЛЬНЫЙ ИСП...","[1125024005920, 1065024016771]","[АО ""ЗАРУБЕЖНЕФТЬ"", ООО ""ЭЛИТ ГРУПП"", ЗАО ""АБМ...","[1047708046870, 1147746842253, 1027700560491, ...","[АО ""РХК"", ООО ""ДНС ГРУПП"", ООО ""ТМК НГС"", ООО...","[1177746123367, 1152543021410, 1086672000030, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,"ООО ГК ""ПОГОДА В ДОМЕ""",1207300012700,Действующая компания,7321013237,732101001,46031649,16 октября 2020,"432048, Ульяновская область, г. о. город Ульян...",Торговля оптовая бытовыми электротоварами,"[464300, 161000, 161010, 161020, 161030, 16109...",...,[],[],[],[],"[АДМИНИСТРАЦИЯ МУНИЦИПАЛЬНОГО ОБРАЗОВАНИЯ ""НОВ...",[1027300786303],[],[],"[ООО ""ИНКОТЕКС-СК"", ООО ""ДРИВИКС"", ООО ""РЕГИОН...","[1047796831037, 1097746843700, 1127746535586, ..."
235,"ООО ""СИНАРА-ГТР ТАГАНРОГ""",1207700463992,Действующая компания,9709067611,615401001,46579100,4 декабря 2020,"347905, Ростовская область, г. о. город Таганр...",Деятельность сухопутного пассажирского транспо...,"[493100, 162300, 232010, 232020, 421100, 42120...",...,[],[],[],[],None,None,None,None,"[МП ""МАГГОРТРАНС"", ООО ""ПИТЕРАВТО"", ЕМУП ""ГОРТ...","[1027402174304, 1037841003298, 1026605253025, ..."
236,"АО ""ОСК""",1026301414930,Действующая компания,6312013969,631701001,04885610,22 июля 2002,"443099, Самарская область, г. о. Самара, вн. р...","Страхование, кроме страхования жизни",[651200],...,[],"[/id/1166313067920-aktivgruppa, /id/1037700051...",[/id/1026300954514-pk-volga],[],"[АО ""ССК"", ГКУ СО ""СТО"", ООО ""СВГК"", ФКУ ""ЦХИС...","[1056367019136, 1066315014952, 1026300892529, ...",[],[],"[ООО СК ""МИРАН"", ООО С

Сохраняем обработанные данные об организациях

In [ ]:
data.to_csv('companies_40.csv')